# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ],
}
"""

In [28]:
# Multi-shot prompting for selecting relevant links
link_system_prompt = """
You are provided with a list of links found on a webpage. 
Your task is to decide which of these links are most relevant for a company brochure. 
Relevant links typically include an About page, Careers page, Products page, or Contact page.
You should always respond in the following JSON format:

Example 1:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about"},
        {"type": "careers page", "url": "https://company.com/careers"}
    ]
}

Example 2:
{
    "links": [
        {"type": "products page", "url": "https://company.com/products"},
        {"type": "contact page", "url": "https://company.com/contact"}
    ]
}

Now, analyze the provided links and generate a response following this format.
"""

In [29]:
print(link_system_prompt)


You are provided with a list of links found on a webpage. 
Your task is to decide which of these links are most relevant for a company brochure. 
Relevant links typically include an About page, Careers page, Products page, or Contact page.
You should always respond in the following JSON format:

Example 1:
{
    "links": [
        {"type": "about page", "url": "https://company.com/about"},
        {"type": "careers page", "url": "https://company.com/careers"}
    ]
}

Example 2:
{
    "links": [
        {"type": "products page", "url": "https://company.com/products"},
        {"type": "contact page", "url": "https://company.com/contact"}
    ]
}

Now, analyze the provided links and generate a response following this format.



In [30]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [31]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [32]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [33]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/manycore-research/SpatialLM-Llama-1B',
 '/ByteDance/MegaTTS3',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/a-m-team/AM-DeepSeek-R1-Distilled-1.4M',
 '/datasets/Anthropic/EconomicIndex',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',

In [34]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'products page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'contact page', 'url': 'https://huggingface.co/contact'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [35]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [36]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'products page', 'url': 'https://huggingface.co/models'}, {'type': 'products page', 'url': 'https://huggingface.co/datasets'}, {'type': 'products page', 'url': 'https://huggingface.co/spaces'}, {'type': 'contact page', 'url': 'https://discuss.huggingface.co'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen2.5-Omni-7B
Updated
3 days ago
•
70.7k
•
1.1k
deepseek-ai/DeepSeek-V3-0324
Updated
7 days ago
•
1

In [ ]:
#system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [37]:
#2nd part of challenge

# Structured prompt for generating the brochure
system_prompt = """
You are an expert in marketing copywriting. Your task is to generate a structured company brochure 
based on the provided company details and selected links. The brochure should be well-organized 
and formatted into clear sections:

**Structure of the brochure:**

1. **Introduction**  
   - Provide a brief overview of the company, summarizing its key highlights.

2. **About Us**  
   - Detail the company's background, mission, and core values.  

3. **Products & Services**  
   - List and describe the main products or services the company offers.  

4. **Careers**  
   - Mention any job opportunities, work culture, and hiring process if applicable.  

5. **Contact Information**  
   - Include ways to reach the company, such as website, email, and phone.  

**Formatting Guidelines:**  
- Use clear section headings (e.g., "About Us", "Products & Services")  
- Write in a professional and engaging tone.  
- Keep the brochure concise yet informative.  

Generate the brochure following this structure.
"""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [38]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [39]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'products page', 'url': 'https://huggingface.co/models'}, {'type': 'products page', 'url': 'https://huggingface.co/datasets'}, {'type': 'products page', 'url': 'https://huggingface.co/spaces'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}]}


# Hugging Face Company Brochure

## Introduction
Welcome to Hugging Face, the AI community building the future. Hugging Face is a pioneering platform where the machine learning community comes together to create, share, and collaborate on models, datasets, and applications. With more than 1 million models and a myriad of datasets, we empower developers, researchers, and innovators alike to accelerate their journey in AI and machine learning.

## About Us
Founded with the mission to make machine learning accessible and collaborative, Hugging Face values openness, community, and innovation. Our platform is built on the belief that collective knowledge can drive progress in artificial intelligence, paving the way for advanced research and real-world applications. We pride ourselves on fostering a supportive and inclusive environment, where collaboration enhances the potential of each member.

## Products & Services
Hugging Face offers a wide range of products and services:

- **Models**: Access over 1 million high-quality machine learning models for various tasks including text, image, and audio processing.
- **Datasets**: Explore and utilize over 250,000 datasets to power your machine learning projects.
- **Spaces**: Collaborate and share applications within our user-friendly environment tailored for AI developers.
- **Enterprise Solutions**: Customized enterprise-grade services that provide advanced security and dedicated support starting at $20/user/month.
- **Open Source Tools**: Contribute to and leverage state-of-the-art tools like Transformers, Diffusers, and Tokenizers for research and production.

## Careers
At Hugging Face, we are always on the lookout for passionate individuals to join our team. We promote a positive work culture that values collaboration, creativity, and continuous learning. If you're interested in being part of an innovative AI community, check out our careers page for current job openings and application details.

## Contact Information
For more information, inquiries, or support, you can reach us at:
- **Website**: [huggingface.co](https://huggingface.co)
- **Email**: support@huggingface.co
- **Phone**: [insert phone number]

Join us in building the future of AI! Together, we can empower the next generation of machine learning innovations.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [40]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [41]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'products page', 'url': 'https://huggingface.co/models'}, {'type': 'products page', 'url': 'https://huggingface.co/datasets'}, {'type': 'contact page', 'url': 'https://huggingface.co/chat'}]}



# Hugging Face Company Brochure

## Introduction
Welcome to Hugging Face, the AI community dedicated to building the future through collaboration and innovation. As a leader in the machine learning community, we offer a platform for individuals and organizations to create, discover, and collaborate on cutting-edge models, datasets, and applications.

## About Us
Hugging Face is on a mission to advance the field of machine learning by fostering collaboration among its members. Our core values revolve around open-source technology, community participation, and accessibility. We believe that together, we can build tools that leverage artificial intelligence for the benefit of all. With over 50,000 organizations leveraging our platform, Hugging Face is setting the pace in the AI space.

## Products & Services
At Hugging Face, we provide a robust range of products and services:

- **Models**: Access a library with over 1 million models including state-of-the-art machine learning tools for PyTorch, TensorFlow, and JAX.
- **Datasets**: Explore and share 250,000+ datasets suitable for various ML tasks, which can enhance your machine learning projects.
- **Spaces**: Create and run applications effortlessly with thousands of ready-to-use ML applications available on our platform.
- **Enterprise Solutions**: Tailored for businesses, our enterprise solutions come with advanced features such as security, compliance, and dedicated support.
  
Whether you are a researcher, developer, or business leader, we have resources to accelerate your machine learning initiatives.

## Careers
Join the Hugging Face community! We are looking for passionate individuals committed to shaping the future of AI. We offer a dynamic work environment that encourages innovation and collaboration. Check our careers page for current job openings and become part of our journey in transforming artificial intelligence.

## Contact Information
Ready to explore more? Reach out to us through the following channels:

- **Website**: [Hugging Face](https://huggingface.co)
- **Email**: contact@huggingface.co
- **Phone**: +1 (800) 123-4567

Connect with us on our social platforms: [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), [Discord](https://discord.gg/huggingface).

---

Take the next step in your AI journey with Hugging Face today!



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

In [49]:
#3rd part of challenge

def translate_brochure(brochure_content, language="Spanish"):
    system_prompt = f"You are a skilled translator. Translate the following brochure text into {language}. \
    Ensure the translation is idiomatic, matching the target language's natural structure, expressions, and nuances. \
    The translated version should feel native and not like a direct translation. \
    Adapt the tone appropriately—marketing language in {language} should be engaging yet culturally suitable. \
    Output the translated brochure in Markdown format. \
    "
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_content}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [50]:
spanish_brochure = translate_brochure("Hugging Face", "Spanish")
print(spanish_brochure)

# Hugging Face

## Bienvenidos a la comunidad de Hugging Face

En Hugging Face, creemos que el desarrollo de la inteligencia artificial debe ser accesible para todos. Nuestra misión es democratizar el acceso y la comprensión de la IA, permitiendo a desarrolladores, investigadores y empresas construir soluciones innovadoras. 

### ¿Qué ofrecemos?

- **Modelos preentrenados**: Descubre una amplia biblioteca de modelos de procesamiento de lenguaje natural que pueden ser adaptados a tus necesidades específicas. Desde traducciones hasta generación de texto, tenemos todo cubierto.

- **Herramientas fáciles de usar**: Con nuestras herramientas, podrás integrar IA en tus proyectos de manera sencilla, sin necesidad de ser un experto. Empezar es muy fácil y rápido.

- **Comunidad activa**: Únete a una vibrante comunidad de entusiastas de la IA. Comparte tus ideas, aprende de otros y colabora en proyectos emocionantes.

### ¿Por qué elegir Hugging Face?

- **Innovación constante**: Estamos a la vanguardia de la investigación en IA y actualizamos regularmente nuestra plataforma para ofrecerte lo último en tecnología.

- **Soporte excepcional**: Nuestro equipo está aquí para ayudarte. Ofrecemos asistencia y recursos para que puedas sacar el máximo provecho de nuestras soluciones.

- **Compromiso con la ética**: Nos tomamos en serio la responsabilidad social y trabajamos con un enfoque ético para garantizar que nuestro trabajo tenga un impacto positivo en la sociedad.

### ¡Únete a nosotros hoy!

Visita nuestra página web y descubre cómo Hugging Face puede transformar tus proyectos con inteligencia artificial. Juntos, ¡haré del mundo un lugar más inteligente y conectado!

---

En Hugging Face, el futuro de la IA está al alcance de tu mano. ¡Empecemos este emocionante viaje juntos!

None


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>